## Адрес с Судебного кабинета

In [75]:
# -*- coding: utf-8 -*-
r"""
Скрипт для подбора адреса и ЧСИ по ИИН:

1) Логинится в office.sud.kz (переключает язык на РУС).
2) Переходит в "Подача документов".
3) Выбирает CIVIL / FIRSTINSTANCE / Иск и нажимает «Отправить».
4) На createRequest.xhtml заполняет поля и открывает форму участника.
5) Берёт ИИН из Excel-файла
      C:\Users\User\Desktop\Новая папка\Список для подачи заявления ЧСИ через АИС ОИП_Omega.xlsx
   (колонка E),
   по каждому ИИН тянет данные:
      - полный адрес ("Место жительства") пишет в колонку I;
      - по адресу подбирает ФИО ЧСИ из файла
        C:\Users\User\Desktop\Новая папка\ЧСИ в АИС ОИП.xlsx
        (район — колонка B, ФИО — колонка C, регион — колонка D)
        и пишет ФИО ЧСИ в колонку H.

Логика парсинга данных с Судебного кабинета сохранена из твоего рабочего скрипта.
"""

import os
import time
import logging
import re
from difflib import SequenceMatcher

from openpyxl import load_workbook

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    ElementNotInteractableException,
)

# ========= КОНФИГ =========
USER_AUTH     = "230240016634"        # твой ИИН/БИН
USER_PASSWORD = "n3y&pAM5mD&4zKZ"     # твой пароль

BASE      = "https://office.sud.kz"
LOGIN_URL = f"{BASE}/index.xhtml"
HOME_URL  = f"{BASE}/form/proceedings/services.xhtml"
SEND_DOCS_URL = f"{BASE}/form/send/index.xhtml"

# Паузы
WAIT   = 1.0

# ========= ПУТИ =========
BASE_DIR   = r"C:\Users\User\Desktop\Новая папка"
INPUT_XLSX = os.path.join(BASE_DIR, "Список для подачи заявления ЧСИ через АИС ОИП_Omega.xlsx")
OUT_XLSX   = INPUT_XLSX  # пишем туда же

CHSI_XLSX  = os.path.join(BASE_DIR, "ЧСИ в АИС ОИП.xlsx")

# Колонки в основном файле (номер столбца)
COL_IIN   = 5  # E – ИИН
COL_CHSI  = 8  # H – ФИО ЧСИ
COL_ADDR  = 9  # I – полный адрес

# ========= ЛОГИ =========
logging.basicConfig(
    filename="sud_chsi_script.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    encoding="utf-8",
)

def log_step(msg):
    print(msg)
    logging.info(msg)

def log_warn(msg):
    print("WARN:", msg)
    logging.warning(msg)

def log_err(msg):
    print("ERROR:", msg)
    logging.error(msg)

def log4(msg):
    print(time.strftime("[%H:%M:%S]"), msg, flush=True)
    logging.info(msg)

# ========= DRIVER =========
def init_driver() -> webdriver.Chrome:
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.page_load_strategy = "eager"

    prefs = {"profile.managed_default_content_settings.images": 2}
    opts.add_experimental_option("prefs", prefs)
    opts.add_argument("--blink-settings=imagesEnabled=false")
    opts.add_argument("--disable-extensions")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-background-networking")
    opts.add_argument("--disable-renderer-backgrounding")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    return driver

# ========= ВСПОМОГАТЕЛЬНЫЕ =========
def find_with_fallbacks(driver, variants, desc, tries=8, delay=0.4):
    """Пробует несколько локаторов с коротким ретраем."""
    last = None
    for _ in range(tries):
        for by, sel in variants:
            try:
                el = driver.find_element(by, sel)
                logging.info(f"Нашёл {desc} по {by}={sel}")
                return el
            except Exception as e:
                last = e
        time.sleep(delay)
    raise NoSuchElementException(f"Не удалось найти {desc}. Последняя ошибка: {last}")

# ========= LOGIN =========
def login(driver):
    log_step("[LOGIN] Открываю страницу логина…")
    driver.get(LOGIN_URL)
    time.sleep(WAIT)

    # язык — РУС
    try:
        ru = driver.find_elements(By.XPATH, "//a[contains(.,'РУС') and not(contains(@class,'active'))]")
        if ru:
            ru[0].click()
            log_step("[LOGIN] Переключил язык на РУС")
            time.sleep(WAIT)
    except Exception:
        log_warn("Не удалось переключить язык на РУС")

    # ЛОГИН
    login_el = find_with_fallbacks(
        driver,
        variants=[
            (By.ID, "j_idt78:auth:xin"),
            (By.XPATH, "//input[contains(@id,':auth:xin')]"),
            (By.NAME, "j_idt78:auth:xin"),
            (By.XPATH, "//input[contains(@placeholder,'ИИН') or contains(@placeholder,'ЖСН') or contains(@placeholder,'ИИН/БИН')]"),
            (By.CSS_SELECTOR, "input[type='email']"),
        ],
        desc="поле ИИН/БИН"
    )
    pass_el = find_with_fallbacks(
        driver,
        variants=[
            (By.ID, "j_idt78:auth:password"),
            (By.XPATH, "//input[contains(@id,':auth:password')]"),
            (By.XPATH, "//input[@type='password']"),
            (By.XPATH, "//input[contains(@placeholder,'Пароль') or contains(@placeholder,'Құпия')]"),
        ],
        desc="поле Пароль"
    )
    submit = find_with_fallbacks(
        driver,
        variants=[
            (By.CSS_SELECTOR, "input.button-primary[type='submit']"),
            (By.XPATH, "//input[@type='submit' and (contains(@value,'Войти') or contains(@class,'button-primary'))]"),
            (By.XPATH, "//button[contains(.,'Войти')]"),
        ],
        desc="кнопка Войти"
    )

    login_el.clear(); login_el.send_keys(USER_AUTH)
    pass_el.clear();  pass_el.send_keys(USER_PASSWORD)
    log_step("[LOGIN] Ввёл ИИН/БИН и Пароль")
    submit.click()
    log_step("[LOGIN] Нажал 'Войти'")
    time.sleep(WAIT * 2)

# ========= ПЕРЕХОД В «Подача документов» =========
def go_to_send_docs(driver):
    log_step("[NAV] Открываю главную страницу (плитки)…")
    driver.get(HOME_URL)
    time.sleep(WAIT)

    link = None
    try:
        link = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/form/send/index.xhtml')]"))
        )
    except TimeoutException:
        link = None

    if not link:
        try:
            link = driver.find_element(By.XPATH, "//p[normalize-space()='Подача документов']/ancestor::a")
        except Exception:
            try:
                link = driver.find_element(By.XPATH, "//a[.//p[contains(normalize-space(),'Подача документ')]]")
            except Exception:
                link = None

    if link:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", link)
        except Exception:
            pass
        try:
            link.click()
        except Exception:
            driver.execute_script("arguments[0].click();", link)
        log_step("[NAV] Кликнул по плитке 'Подача документов'")
    else:
        log_warn("[NAV] Плитка не найдена, открываю 'Подача документов' по прямому URL")
        driver.get(SEND_DOCS_URL)

    try:
        WebDriverWait(driver, 10).until(EC.url_contains("/form/send"))
    except TimeoutException:
        log_warn("[NAV] Не удалось явно подтвердить загрузку страницы 'Подача документов'")

def _css_from_locator(locator):
    by, sel = locator
    if by == By.CSS_SELECTOR:
        return sel
    if by == By.ID:
        css_id = sel.replace(":", r"\:")
        return f"#{css_id}"
    return sel

def _select_by_value_robust(driver, locator, value, desc, timeout=20, tries=3):
    w = WebDriverWait(driver, timeout)

    # ждём <select>
    w.until(EC.presence_of_element_located(locator))

    # ждём наличия нужной опции
    def _opt_present(d):
        try:
            el = d.find_element(*locator)
            return any((o.get_attribute("value") or "") == value
                       for o in el.find_elements(By.TAG_NAME, "option"))
        except StaleElementReferenceException:
            return False
    w.until(_opt_present)

    last_err = None
    for _ in range(tries):
        try:
            el = driver.find_element(*locator)
            Select(el).select_by_value(value)
            w.until(lambda d: d.find_element(*locator).get_attribute("value") == value)
            print(f"✔ {desc} → {value}")
            return
        except (StaleElementReferenceException, TimeoutException) as e:
            last_err = e

    # JS-фолбэк
    ok = driver.execute_script("""
        var el = document.querySelector(arguments[0]);
        if(!el) return false;
        var idx = Array.from(el.options).findIndex(o => (o.value||'') === arguments[1]);
        if(idx < 0) return false;
        el.selectedIndex = idx;
        el.dispatchEvent(new Event('change', {bubbles:true}));
        return true;
    """, _css_from_locator(locator), value)

    if not ok:
        raise TimeoutException(f"Не удалось выбрать '{desc}' со значением {value}. Последняя ошибка: {last_err}")
    WebDriverWait(driver, 10).until(lambda d: d.find_element(*locator).get_attribute("value") == value)
    print(f"✔ {desc} → {value} (через JS)")

def send_claim(driver, timeout=20):
    """
    На странице /form/send/index.xhtml:
      1) Тип производства = CIVIL
      2) Инстанция = FIRSTINSTANCE
      3) Тип документа = Иск (3)
      4) Нажать «Отправить»
    """
    w = WebDriverWait(driver, timeout)

    _select_by_value_robust(
        driver, (By.CSS_SELECTOR, "select[id$=':case-type']"),
        "CIVIL", "Тип производства", timeout=timeout
    )

    _select_by_value_robust(
        driver, (By.CSS_SELECTOR, "select[id$=':instance']"),
        "FIRSTINSTANCE", "Инстанция", timeout=timeout
    )

    _select_by_value_robust(
        driver, (By.CSS_SELECTOR, "select[id$=':request']"),
        "3", "Тип документа", timeout=timeout
    )

    submit_locators = [
        (By.XPATH, "//input[@type='submit' and @value='Отправить']"),
        (By.XPATH, "//button[normalize-space()='Отправить']"),
        # на всякий случай — казахский вариант
        (By.XPATH, "//input[@type='submit' and @value='Жіберу']"),
        (By.XPATH, "//button[normalize-space()='Жіберу']"),
        # и любой submit внутри формы подачи документов
        (By.XPATH, "//form[contains(@action,'send') or contains(@id,'sendForm')]//input[@type='submit']"),
    ]
    btn = None
    for loc in submit_locators:
        try:
            btn = w.until(EC.element_to_be_clickable(loc))
            break
        except TimeoutException:
            continue
    if not btn:
        raise TimeoutException("Кнопка «Отправить» не найдена.")

    try:
        btn.click()
    except Exception:
        driver.execute_script("arguments[0].click();", btn)
    print("✔ Нажал «Отправить».")

# ========= БЛОК 3: createRequest + форма участника =========
def _wait(drv, cond, t=25):
    return WebDriverWait(drv, t, ignored_exceptions=(StaleElementReferenceException,)).until(cond)

def _select_value(drv, css, value, desc, t=25):
    sel = _wait(drv, EC.presence_of_element_located((By.CSS_SELECTOR, css)), t)
    for _ in range(3):
        try:
            Select(sel).select_by_value(value)
            _wait(drv, lambda d: d.find_element(By.CSS_SELECTOR, css).get_attribute("value") == value, 10)
            print(f"✔ {desc} → value={value}")
            return
        except StaleElementReferenceException:
            sel = drv.find_element(By.CSS_SELECTOR, css)
        except Exception:
            pass
    # JS-фолбэк
    drv.execute_script("""
        var s = document.querySelector(arguments[0]);
        var val = arguments[1];
        if(!s) return false;
        s.value = val;
        s.dispatchEvent(new Event('change', {bubbles:true}));
        return true;
    """, css, value)
    _wait(drv, lambda d: d.find_element(By.CSS_SELECTOR, css).get_attribute("value") == value, 10)
    print(f"✔ {desc} → value={value} (JS)")

def _click(drv, locator, desc, t=25):
    btn = _wait(drv, EC.element_to_be_clickable(locator), t)
    drv.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    try:
        btn.click()
    except Exception:
        drv.execute_script("arguments[0].click();", btn)
    print(f"✔ Нажал {desc}")

def block3_fill_case_and_open_modal(driver, timeout=40):
    w = WebDriverWait(driver, timeout)

    # убеждаемся, что на createRequest.xhtml
    try:
        w.until(
            lambda d: "/form/requestType2/createRequest.xhtml" in d.current_url
            or d.find_element(By.CSS_SELECTOR, "select[id$=':edit-category']")
        )
    except Exception:
        raise TimeoutException("Страница createRequest.xhtml не открыта")

    # 1) Вид производства по делу: value=2
    _select_value(
        driver,
        "select[id$=':edit-categoryGroup']",
        "2",
        "Вид производства по делу",
        t=timeout,
    )

    # 2) Категория дела: value=27
    _wait(
        driver,
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "select[id$=':edit-category'] option[value='27']")
        ),
        30,
    )
    _select_value(
        driver,
        "select[id$=':edit-category']",
        "27",
        "Категория дела",
        t=timeout,
    )

    # 3) Характер заявления: value=1
    _wait(
        driver,
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "select[id$=':edit-character'] option[value='1']")
        ),
        20,
    )
    _select_value(
        driver,
        "select[id$=':edit-character']",
        "1",
        "Характер заявления",
        t=timeout,
    )

    # 4) Галочка «Дело упрощенного производства»
    try:
        cb = w.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "input[id$=':edit-simpleProcess'][type='checkbox']")
            )
        )
        if not cb.is_selected():
            driver.execute_script("arguments[0].click();", cb)
        print("✔ Галочка «Дело упрощенного производства» установлена")
    except Exception:
        print("⚠️ Не удалось установить галочку «Дело упрощенного производства» (продолжаю)")

    # 5) Открыть модал «Добавить участника процесса»
    add_locators = [
        (By.XPATH, "//button[contains(.,'Добавить участника процесса')]"),
        (By.CSS_SELECTOR, "button[id*='addPerson']"),
    ]
    next_xpath = (
        "//div[contains(@class,'modal') or contains(@class,'modal-dialog')]"
        "//input[@type='button' or @type='submit'][@value='Далее']"
    )
    for loc in add_locators:
        try:
            _click(driver, loc, "«Добавить участника процесса»", t=timeout)
            _wait(driver, EC.presence_of_element_located((By.XPATH, next_xpath)), 20)
            break
        except TimeoutException:
            continue
    else:
        raise TimeoutException("Кнопка «Добавить участника процесса» не найдена")

    # 6) В модалке жмём «Далее» и ждём поле ИИН
    person_iin_css = "input[id$=':person-iin']"

    def modal_switched(d):
        try:
            return d.find_element(By.CSS_SELECTOR, person_iin_css)
        except Exception:
            return False

    for attempt in range(3):
        try:
            btn_next = _wait(
                driver,
                EC.element_to_be_clickable((By.XPATH, next_xpath)),
                15,
            )

            driver.execute_script(
                "arguments[0].scrollIntoView({block:'center'});", btn_next
            )

            try:
                btn_next.click()
            except Exception:
                driver.execute_script("arguments[0].click();", btn_next)

            _wait(driver, modal_switched, 25)
            print("✅ Блок 3 завершён: открыта форма участника с полем ИИН.")
            break

        except StaleElementReferenceException:
            print("⚠ Stale element на кнопке «Далее», пробую ещё раз…")
            if attempt == 2:
                print("❌ Не удалось нажать «Далее» — кнопка всё время устаревает")
                raise
            time.sleep(1)

# ========= БЛОК 4: парсинг ИИН из Excel (оставляем как было, но под новый файл) =========
print("=== BLOCK-4 for CHSI loaded (IIN from column E, address->I, CHSI->H) ===")

AUTOSAVE_EVERY = 25  # каждые N обработанных ИИН автосейв

# профиль по времени
PERSON_TIMEOUT         = 8
RETRIES_PER_PERSON     = 2
PAUSE_BETWEEN_TRIES    = 0.4
COOLDOWN_BETWEEN_ROWS  = 1.1
MIN_SECONDS_PER_ROW    = 1.2
POLL                   = 0.10
MIN_STABLE_SEC         = 0.35
HARD_WAIT_AFTER_SEARCH = 0.20

# локаторы формы «Добавить участника процесса»
IIN  = (By.CSS_SELECTOR, 'input[id$=":person-iin"]')
SUR  = (By.CSS_SELECTOR, 'input[id$=":person-surname"]')
NAM  = (By.CSS_SELECTOR, 'input[id$=":person-firstname"]')
PATR = (By.CSS_SELECTOR, 'input[id$=":person-patronymic"]')
LIVE = (By.CSS_SELECTOR, 'textarea[id$=":person-livePlace"], input[id$=":person-livePlace"]')

# кнопки/диалоги для перезапуска формы
BTN_CLOSE_PERSON = (
    By.XPATH,
    "//input[@value='Закрыть' and contains(@onclick,'hideFizModalDialog')]"
    " | //input[@id='j_idt278:j_idt328']"
)
BTN_ADD_PERSON = (
    By.XPATH,
    "//button[contains(@onclick,'renderAddPersonModalDialog') or contains(.,'Добавить участника процесса')]"
)
BTN_NEXT_SIDE = (
    By.XPATH,
    "//div[contains(@class,'modal')]//input[@type='button' and @value='Далее'] "
    "| //input[@id='j_idt185:j_idt205']"
)

def _norm_iin(val) -> str:
    if val is None:
        return ""
    s = str(val).strip()
    try:
        if "e" in s.lower():
            s = str(int(float(s)))
    except Exception:
        pass
    s = "".join(ch for ch in s if ch.isdigit())
    return s[:12]

def _get_val(drv, loc) -> str:
    try:
        el = drv.find_element(*loc)
        return (el.get_attribute("value") or "").strip()
    except Exception:
        return ""

def _rf_queue_size(drv):
    try:
        return drv.execute_script("""
            try {
              if (window.RichFaces && RichFaces.Queue && typeof RichFaces.Queue.getSize==='function'){
                return RichFaces.Queue.getSize();
              }
            } catch(e){}
            return 0;
        """) or 0
    except Exception:
        return 0

def _wait_queue_quiet(drv, min_quiet=MIN_STABLE_SEC, total=6):
    t_end = time.time() + total
    quiet_start = None
    last = -1
    while time.time() < t_end:
        q = _rf_queue_size(drv)
        if q != last:
            last = q
            log4(f"   ajax queue size = {q}")
        if q == 0:
            if quiet_start is None:
                quiet_start = time.time()
            if time.time() - quiet_start >= min_quiet:
                return True
        else:
            quiet_start = None
        time.sleep(0.15)
    return False

def _values_snapshot(drv):
    return {
        "sur":  _get_val(drv, SUR),
        "name": _get_val(drv, NAM),
        "patr": _get_val(drv, PATR),
        "live": _get_val(drv, LIVE),
    }

def _stable_after_change(drv, before: dict, timeout=PERSON_TIMEOUT):
    w = WebDriverWait(
        drv,
        timeout,
        poll_frequency=POLL,
        ignored_exceptions=(StaleElementReferenceException,),
    )

    def _changed(_):
        now = _values_snapshot(drv)
        changed = any(now[k] != before.get(k, "") for k in now) and any(now.values())
        return now if changed else False

    try:
        changed_vals = w.until(_changed)
        t0 = time.time()
        last = changed_vals
        while time.time() - t0 < MIN_STABLE_SEC:
            time.sleep(0.15)
            now = _values_snapshot(drv)
            if now != last:
                t0 = time.time()
                last = now
        return last
    except TimeoutException:
        return _values_snapshot(drv)

def _click_magnifier(drv):
    # сначала пробуем js-функцию
    try:
        drv.execute_script("if (typeof fillPersonData==='function'){fillPersonData('j_idt278:person-iin');}")
        log4("   вызвал fillPersonData()")
        return True
    except Exception:
        pass

    # потом ищем иконку-лупу
    try:
        span = drv.find_element(
            By.XPATH,
            "//input[contains(@id,':person-iin')]/following-sibling::span[contains(@class,'gbdSearch')]",
        )
        drv.execute_script("arguments[0].scrollIntoView({block:'center'});", span)
        span.click()
        log4("   клик по иконке-лупе")
        return True
    except Exception:
        try:
            span = drv.find_element(By.CSS_SELECTOR, "span.gbdSearch")
            drv.execute_script("arguments[0].scrollIntoView({block:'center'});", span)
            span.click()
            log4("   клик по .gbdSearch (fallback)")
            return True
        except Exception:
            log4("   ⚠ не смог нажать лупу")
            return False

def _trigger_and_wait(drv) -> dict:
    before = _values_snapshot(drv)
    for attempt in range(1, RETRIES_PER_PERSON + 1):
        log4(f"   попытка #{attempt} — запуск поиска")
        _click_magnifier(drv)
        time.sleep(HARD_WAIT_AFTER_SEARCH)
        _wait_queue_quiet(drv, min_quiet=MIN_STABLE_SEC, total=PERSON_TIMEOUT)
        after = _stable_after_change(drv, before, timeout=PERSON_TIMEOUT)
        changed = any(after[k] != before.get(k, "") for k in after) and any(after.values())
        log4(f"   снэпшот: {after}  | changed={changed}")
        if changed:
            return after
        log4("   ↺ не изменилось — подожду и попробую ещё раз")
        time.sleep(PAUSE_BETWEEN_TRIES)
    return {"sur": "", "name": "", "patr": "", "live": ""}

def _safe_click(drv, loc, desc=""):
    try:
        el = WebDriverWait(drv, 6).until(EC.element_to_be_clickable(loc))
    except TimeoutException:
        try:
            el = drv.find_element(*loc)
        except Exception:
            return False
    try:
        drv.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        el.click()
        return True
    except Exception:
        try:
            drv.execute_script("arguments[0].click();", el)
            return True
        except Exception:
            return False

def _reopen_person_modal(drv):
    """
    Закрыть форму участника, снова нажать «Добавить участника процесса»
    и (если нужно) «Далее» в выборе стороны.
    """
    log4("   ↻ перезапуск формы участника…")

    _safe_click(drv, BTN_CLOSE_PERSON, "Закрыть")
    time.sleep(0.7)

    if not _safe_click(drv, BTN_ADD_PERSON, "Добавить участника"):
        log4("   ⚠ не нашёл кнопку «Добавить участника процесса»")
        return False

    clicked_next = _safe_click(drv, BTN_NEXT_SIDE, "Далее")

    try:
        WebDriverWait(drv, 12).until(EC.presence_of_element_located(IIN))
        if clicked_next:
            log4("   форма участника открыта заново (после выбора стороны)")
        else:
            log4("   форма участника открыта заново (без выбора стороны)")
        return True
    except TimeoutException:
        log4("   ⚠ не дождался поля ИИН после перезапуска")
        return False

def ensure_person_form_open(drv, timeout=15):
    """
    Гарантируем, что открыта форма участника с полем ИИН.
    Если поля нет — пробуем заново открыть модалку.
    """
    try:
        return WebDriverWait(drv, timeout).until(
            EC.presence_of_element_located(IIN)
        )
    except TimeoutException:
        if not _reopen_person_modal(drv):
            raise RuntimeError(
                "Не удалось найти форму участника (поле ИИН). "
                "Возможно, заявка закрылась или сессия слетела."
            )
        return WebDriverWait(drv, timeout).until(
            EC.presence_of_element_located(IIN)
        )

def _safe_save(wb, path):
    try:
        wb.save(path)
        log4(f"💾 autosave to {path}")
        return True
    except Exception as e:
        log4(f"⚠ не удалось сохранить файл: {e}")
        return False

# ===== ЧТЕНИЕ СПРАВОЧНИКА ЧСИ И "УМНОЕ" СОВПАДЕНИЕ =====
def _normalize_txt(s: str) -> str:
    if not s:
        return ""
    s = str(s).upper()
    s = s.replace("Ё", "Е")
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def fio_title_case(s: str) -> str:
    """Нормальное отображение ФИО (каждое слово и часть через дефис с заглавной)."""
    if not s:
        return ""
    s = re.sub(r"\s+", " ", str(s).strip())
    words = []
    for part in s.split(" "):
        sub = [sp.capitalize() for sp in part.split("-")]
        words.append("-".join(sub))
    return " ".join(words)

def load_chsi_table(path: str):
    rows = []
    if not os.path.exists(path):
        log_warn(f"[CHSI] Файл не найден: {path}")
        return rows

    wb = load_workbook(path, data_only=True, read_only=True)
    ws = wb.active
    for row in ws.iter_rows(min_row=2, values_only=True):
        district = row[1] if len(row) > 1 else None   # колонка B
        fio      = row[2] if len(row) > 2 else None   # колонка C
        region   = row[3] if len(row) > 3 else None   # колонка D

        if not fio:
            continue

        reg_u = _normalize_txt(region)
        dist_u = _normalize_txt(district)
        fio_u = _normalize_txt(fio)

        rows.append({
            "district": district or "",
            "region": region or "",
            "fio": fio or "",
            "district_u": dist_u,
            "region_u": reg_u,
            "fio_u": fio_u,
        })
    wb.close()
    log_step(f"[CHSI] Загружено строк: {len(rows)}")
    return rows

CHSI_ROWS = load_chsi_table(CHSI_XLSX)

# Разрешённые ЧСИ для Карагандинской области
KARAGANDA_CHSI_ALLOWED = {
    _normalize_txt("Алибеков Арман Жасуланович"),
    _normalize_txt("Балтабаев Аян Хайдарович"),
    _normalize_txt("Висмурадов Хусайн Салманович"),
    _normalize_txt("Келдибеков Жазибек Мажирович"),
}

def _find_chsi_by_fio(fio_search: str):
    """Поиск ЧСИ по ФИО в справочнике."""
    if not fio_search:
        return None
    target = _normalize_txt(fio_search)
    for r in CHSI_ROWS:
        if _normalize_txt(r.get("fio", "")) == target:
            return r
    return None

def pick_chsi_by_address(address: str):
    """
    Подбор ЧСИ по адресу.

    1) Сначала применяем жёсткие правила по началу адреса (АЛМАТЫ, АСТАНА,
       ШЫМКЕНТ, все области и детальная логика по Карагандинской области).
    2) Если ни одно правило не сработало — используем старую "умную" логику
       по региону/району, но для КАРАГАНДИНСКОЙ ОБЛАСТИ ограничиваемся
       только 4 разрешёнными ЧСИ.
    """
    if not address:
        return None, 0.0

    addr_u = _normalize_txt(address)

    # --------- ЖЁСТКИЕ ПРАВИЛА ПО ГОРОДАМ-РЕСПУБЛИКАНСКОГО ЗНАЧЕНИЯ ---------
    if addr_u.startswith("КАЗАХСТАН, АЛМАТЫ"):
        return _find_chsi_by_fio("Нугманова Ильнара Султанбековна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, АСТАНА") or addr_u.startswith("КАЗАХСТАН, НУР-СУЛТАН"):
        return _find_chsi_by_fio("Буганаев Уалихан Муратбекович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ШЫМКЕНТ"):
        return _find_chsi_by_fio("Бейсенова Алия Жумабековна"), 100.0

    # --------- ЖЁСТКИЕ ПРАВИЛА ПО ОБЛАСТЯМ ---------
    if addr_u.startswith("КАЗАХСТАН, АКМОЛИНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Гафурова Зарина Радиковна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, АКТЮБИНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Кулшанов Алмас Бакитович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, АЛМАТИНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Досымбекова Эльмира Есбосыновна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, АТЫРАУСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Мамбетова Айгуль Махсутовна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Жайсанбаев Куаныш Максатович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ЖАМБЫЛСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Болтаев Еркебулан Бекайдарович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Сюнтиев Адиль Сапарович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, КОСТАНАЙСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Исмухамедов Анвар Маликович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Алшагиров Ербулат Қайырбекұлы"), 100.0

    if addr_u.startswith("КАЗАХСТАН, МАНГИСТАУСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Смаилов Рауан Темиргалиевич"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ОБЛАСТЬ АБАЙ"):
        return _find_chsi_by_fio("Бектемиров Рахат Хамитович"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ОБЛАСТЬ ЖЕТІСУ"):
        return _find_chsi_by_fio("Асбекова Асем Сакеновна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ОБЛАСТЬ ҰЛЫТАУ"):
        return _find_chsi_by_fio("Длимбетов Ардак Катарбаевич"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ПАВЛОДАРСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Шахматалинова Айжан Айдосовна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Темирханова Дина Куандыковна"), 100.0

    if addr_u.startswith("КАЗАХСТАН, ТУРКЕСТАНСКАЯ ОБЛАСТЬ"):
        return _find_chsi_by_fio("Сердалиев Ерланбек Навбатиллаевич"), 100.0

    # --------- ДЕТАЛЬНАЯ ЛОГИКА ПО КАРАГАНДИНСКОЙ ОБЛАСТИ ---------
    if addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, КАРАГАНДА"):
        return _find_chsi_by_fio("Алибеков Арман Жасуланович"), 100.0

    if (
        addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, ТЕМИРТАУ")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, НУРИНСКИЙ РАЙОН")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, ОСАКАРОВСКИЙ РАЙОН")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, КАРКАРАЛИНСКИЙ РАЙОН")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, БУХАР-ЖЫРАУСКИЙ Р-Н")
    ):
        return _find_chsi_by_fio("Балтабаев Аян Хайдарович"), 100.0

    if (
        addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, АБАЙСКИЙ РАЙОН")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, ШАХТИНСК")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, САРАНЬ")
    ):
        return _find_chsi_by_fio("Висмурадов Хусайн Салманович"), 100.0

    if (
        addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, БАЛХАШ")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, ПРИОЗЕРСК")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, АКТОГАЙСКИЙ РАЙОН")
        or addr_u.startswith("КАЗАХСТАН, КАРАГАНДИНСКАЯ ОБЛАСТЬ, ШЕТСКИЙ РАЙОН")
    ):
        return _find_chsi_by_fio("Келдибеков Жазибек Мажирович"), 100.0

    # --------- ДАЛЬШЕ — СТАРАЯ "УМНАЯ" ЛОГИКА (ФОЛБЭК) ---------

    best = None
    best_score = 0.0

    # если в адресе Карагандинская область — ограничиваемся 4 ЧСИ
    if "КАРАГАНДИНСКАЯ ОБЛАСТЬ" in addr_u:
        candidates = [r for r in CHSI_ROWS if r.get("fio_u") in KARAGANDA_CHSI_ALLOWED]
        if not candidates:
            candidates = CHSI_ROWS  # на всякий случай
    else:
        candidates = CHSI_ROWS

    for r in candidates:
        score = 0.0
        reg_u = r.get("region_u", "")
        dist_u = r.get("district_u", "")

        if reg_u and reg_u in addr_u:
            score += 2.0
        if dist_u and dist_u in addr_u:
            score += 3.0

        if dist_u:
            ratio = SequenceMatcher(None, dist_u, addr_u).ratio()
            score += ratio  # 0..1

        if score > best_score:
            best_score = score
            best = r

    return (best, best_score) if best else (None, 0.0)


# ===== ОСНОВНОЙ ЦИКЛ ПО ИИН (НОВАЯ ВЕРСИЯ) =====
def parse_chsi_from_excel(drv):
    """
    Новый вариант:
      - ИИН берём из колонки E (5);
      - по каждому ИИН тянем данные с сайта;
      - полный адрес пишем в колонку I (9);
      - по адресу подбираем ЧСИ из CHSI_XLSX и пишем ФИО в колонку H (8).
    """
    MAX_RESTARTS_PER_ROW = 2

    log4("=== START parse_chsi_from_excel ===")
    log4(f"Файл: {INPUT_XLSX}")

    wb = load_workbook(INPUT_XLSX, data_only=True)
    ws = wb.active

    # Заголовки H/I
    header_chsi = ws.cell(row=1, column=COL_CHSI)
    if not header_chsi.value or str(header_chsi.value).strip() == "":
        header_chsi.value = "ФИО ЧСИ (подбор)"

    header_addr = ws.cell(row=1, column=COL_ADDR)
    if not header_addr.value or str(header_addr.value).strip() == "":
        header_addr.value = "Адрес с Судебного кабинета"

    WebDriverWait(drv, 20).until(EC.presence_of_element_located(IIN))

    processed = 0
    failed_rows = []

    try:
        for r in range(2, ws.max_row + 1):
            src_iin = _norm_iin(ws.cell(row=r, column=COL_IIN).value)
            if not src_iin or len(src_iin) != 12:
                continue

            processed += 1
            t_row_start = time.time()
            log4(f"#{processed} (Excel row {r}) → ИИН: {src_iin}")

            success = False

            for restart in range(MAX_RESTARTS_PER_ROW + 1):
                if restart > 0:
                    log4(f"   ↻ полный перезапуск формы для этого ИИН (рестарт {restart})")
                    if not _reopen_person_modal(drv):
                        log4("   ❌ не удалось перезапустить форму, выхожу из попыток по этому ИИН")
                        break

                try:
                    iin_el = ensure_person_form_open(drv, timeout=20)
                except RuntimeError as e:
                    log4(f"   ❌ форма участника недоступна: {e}")
                    break

                drv.execute_script("arguments[0].scrollIntoView({block:'center'});", iin_el)

                # очищаем поле
                try:
                    iin_el.clear()
                except Exception:
                    try:
                        drv.execute_script("arguments[0].value='';", iin_el)
                    except Exception:
                        pass

                # вводим ИИН
                try:
                    iin_el.send_keys(src_iin)
                except ElementNotInteractableException:
                    log4("   ⚠ элемент ИИН не интерактивен — ставлю значение через JS")
                    drv.execute_script("arguments[0].value = arguments[1];", iin_el, src_iin)

                # триггеры change/blur
                try:
                    drv.execute_script(
                        "arguments[0].dispatchEvent(new Event('change',{bubbles:true}));",
                        iin_el,
                    )
                    drv.execute_script(
                        "arguments[0].dispatchEvent(new Event('blur',{bubbles:true}));",
                        iin_el,
                    )
                except Exception:
                    pass

                before = _values_snapshot(drv)
                data = _trigger_and_wait(drv)
                changed = any(data.values()) and any(data[k] != before.get(k, "") for k in data)
                log4(f"   changed={changed}, data={data}")

                if changed:
                    live = data.get("live", "")
                    ws.cell(row=r, column=COL_ADDR, value=live)

                    chsi_row, score = pick_chsi_by_address(live)
                    if chsi_row:
                        fio = fio_title_case(chsi_row["fio"])
                        ws.cell(row=r, column=COL_CHSI, value=fio)
                        log4(f"   → CHSI: '{fio}' (score={score:.3f})")
                    else:
                        log4("   → CHSI: не найден по адресу")

                    success = True
                    break
                else:
                    log4("   данных нет / не обновились, пробую перезапустить форму…")

            if not success:
                failed_rows.append(r)
                log4(f"   → row {r}: ERROR_NO_DATA")

            # паузы и очередь ajax
            _wait_queue_quiet(drv, min_quiet=MIN_STABLE_SEC, total=4)
            time.sleep(COOLDOWN_BETWEEN_ROWS)
            elapsed = time.time() - t_row_start
            if elapsed < MIN_SECONDS_PER_ROW:
                time.sleep(MIN_SECONDS_PER_ROW - elapsed)

            if processed % AUTOSAVE_EVERY == 0:
                _safe_save(wb, OUT_XLSX)

        _safe_save(wb, OUT_XLSX)
        log4(f"✅ Первый проход завершён. Сохранено в: {OUT_XLSX}")

        # Второй проход по проблемным (если нужны, можно оставить / убрать)
        if failed_rows:
            log4(f"=== SECOND PASS: {len(failed_rows)} строк с ERROR_NO_DATA ===")
            try:
                go_to_send_docs(drv)
                send_claim(drv)
                block3_fill_case_and_open_modal(drv)
                WebDriverWait(drv, 20).until(EC.presence_of_element_located(IIN))
            except Exception as e:
                log4(f"⚠ не удалось заново открыть форму участника: {e}")
            else:
                for r in failed_rows:
                    src_iin = _norm_iin(ws.cell(row=r, column=COL_IIN).value)
                    if not src_iin:
                        continue

                    log4(f"[2ND PASS] row {r} → ИИН: {src_iin}")

                    try:
                        iin_el = ensure_person_form_open(drv, timeout=20)
                    except RuntimeError as e:
                        log4(f"   ❌ форма участника недоступна при втором проходе: {e}")
                        break

                    drv.execute_script("arguments[0].scrollIntoView({block:'center'});", iin_el)

                    try:
                        iin_el.clear()
                    except Exception:
                        try:
                            drv.execute_script("arguments[0].value='';", iin_el)
                        except Exception:
                            pass

                    try:
                        iin_el.send_keys(src_iin)
                    except ElementNotInteractableException:
                        log4("   ⚠ [2ND PASS] элемент ИИН не интерактивен — ставлю значение через JS")
                        drv.execute_script("arguments[0].value = arguments[1];", iin_el, src_iin)

                    try:
                        drv.execute_script(
                            "arguments[0].dispatchEvent(new Event('change',{bubbles:true}));",
                            iin_el,
                        )
                        drv.execute_script(
                            "arguments[0].dispatchEvent(new Event('blur',{bubbles:true}));",
                            iin_el,
                        )
                    except Exception:
                        pass

                    before = _values_snapshot(drv)
                    data = _trigger_and_wait(drv)
                    changed = any(data.values()) and any(data[k] != before.get(k, "") for k in data)
                    log4(f"   [2ND PASS] changed={changed}, data={data}")

                    if changed:
                        live = data.get("live", "")
                        ws.cell(row=r, column=COL_ADDR, value=live)

                        chsi_row, score = pick_chsi_by_address(live)
                        if chsi_row:
                            fio = fio_title_case(chsi_row["fio"])
                            ws.cell(row=r, column=COL_CHSI, value=fio)
                            log4(f"   [2ND PASS] → CHSI: '{fio}' (score={score:.3f})")
                        else:
                            log4("   [2ND PASS] → CHSI: не найден")

                    else:
                        log4("   [2ND PASS] снова нет данных, оставляю пустым")

                    _wait_queue_quiet(drv, min_quiet=MIN_STABLE_SEC, total=4)
                    time.sleep(COOLDOWN_BETWEEN_ROWS)

                _safe_save(wb, OUT_XLSX)
                log4(f"✅ Второй проход завершён. Итог: {OUT_XLSX}")

    finally:
        try:
            _safe_save(wb, OUT_XLSX)
        except Exception:
            pass

# ========= ORCHESTRATOR =========
def main():
    drv = init_driver()
    try:
        login(drv)
        go_to_send_docs(drv)
        send_claim(drv)
        block3_fill_case_and_open_modal(drv)
        parse_chsi_from_excel(drv)
    finally:
        # при желании можно закрывать браузер
        # drv.quit()
        log_step("[EXIT] Скрипт завершён.")

if __name__ == "__main__":
    main()


=== BLOCK-4 for CHSI loaded (IIN from column E, address->I, CHSI->H) ===
[CHSI] Загружено строк: 23
[LOGIN] Открываю страницу логина…
[LOGIN] Переключил язык на РУС
[LOGIN] Ввёл ИИН/БИН и Пароль
[LOGIN] Нажал 'Войти'
[NAV] Открываю главную страницу (плитки)…
[NAV] Кликнул по плитке 'Подача документов'
✔ Тип производства → CIVIL
✔ Инстанция → FIRSTINSTANCE
✔ Тип документа → 3
✔ Нажал «Отправить».
✔ Вид производства по делу → value=2
✔ Категория дела → value=27
✔ Характер заявления → value=1
✔ Галочка «Дело упрощенного производства» установлена
✔ Нажал «Добавить участника процесса»
⚠ Stale element на кнопке «Далее», пробую ещё раз…
✅ Блок 3 завершён: открыта форма участника с полем ИИН.
[14:25:26] === START parse_chsi_from_excel ===
[14:25:26] Файл: C:\Users\User\Desktop\Новая папка\Список для подачи заявления ЧСИ через АИС ОИП_Omega.xlsx
[14:25:26] #1 (Excel row 2) → ИИН: 060427600795
[14:25:26]    попытка #1 — запуск поиска
[14:25:26]    вызвал fillPersonData()
[14:25:27]    ajax queue

## Заявления ЧСИ

In [ ]:
# -*- coding: utf-8 -*-
import os
from pathlib import Path
from datetime import datetime

from openpyxl import load_workbook
from docx import Document
from docx2pdf import convert   # pip install docx2pdf

# ---------------- НАСТРОЙКИ ----------------
EXCEL_PATH   = r"C:\Users\User\Desktop\Новая папка\Список для подачи заявления ЧСИ через АИС ОИП_Omega.xlsx"
TEMPLATE_DOC = r"C:\Users\User\Desktop\Новая папка\Шаблон заявления для ЧСИ.docx"
OUT_FOLDER   = r"C:\Users\User\Desktop\Новая папка\Заявления"

# -------------------------------------------------
# Вспомогательные функции
# -------------------------------------------------
def nice_case(s: str) -> str:
    """Первая буква каждого слова заглавная, остальные строчные."""
    if not s:
        return ""
    parts = str(s).split()
    return " ".join(p.capitalize() for p in parts)

def replace_placeholders_in_paragraph(paragraph, mapping: dict):
    """
    ВАЖНО: работаем с целым paragraph.text,
    а потом пересобираем абзац, чтобы плейсхолдеры,
    разрезанные на несколько run'ов, тоже заменялись.
    """
    text = paragraph.text
    changed = False

    for key, value in mapping.items():
        if key in text:
            text = text.replace(key, value)
            changed = True

    if changed:
        if paragraph.runs:
            paragraph.runs[0].text = text
            for r in paragraph.runs[1:]:
                r.text = ""
        else:
            paragraph.add_run(text)

def fill_document(row_dict: dict) -> Document:
    """Подставляет данные из строки Excel в шаблон DOCX и возвращает готовый Document."""
    doc = Document(TEMPLATE_DOC)

    mapping = {
        "{Уникальный номер}": str(row_dict.get("Уникальный номер", "")),
        "{ФИО}": nice_case(row_dict.get("ФИО", "")),
        "{ИИН}": str(row_dict.get("ИИН", "")),
        "{Остаток задолженности}": str(row_dict.get("Остаток задолженности", "")),
        "{ЧСИ}": nice_case(row_dict.get("ЧСИ", "")),
        "{ProjectName}": str(row_dict.get("Продукт", "")),
        "{CurrDate}": datetime.now().strftime("%d.%m.%Y"),
    }

    # Абзацы обычные
    for p in doc.paragraphs:
        replace_placeholders_in_paragraph(p, mapping)

    # Абзацы внутри таблиц (на всякий случай)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    replace_placeholders_in_paragraph(p, mapping)

    return doc

# -------------------------------------------------
# Основная логика
# -------------------------------------------------
def main():
    os.makedirs(OUT_FOLDER, exist_ok=True)

    wb = load_workbook(EXCEL_PATH, read_only=True, data_only=True)
    ws = wb.active

    # Заголовки колонок (первая строка)
    headers = [str(c.value).strip() if c.value is not None else "" for c in ws[1]]

    for row in ws.iter_rows(min_row=2, values_only=True):
        if all(v is None for v in row):
            continue

        row_dict = dict(zip(headers, row))

        fio = nice_case(row_dict.get("ФИО", ""))
        iin = str(row_dict.get("ИИН", "")).strip()

        if not fio or not iin:
            # если нет ФИО или ИИН – пропускаем
            continue

        # 1) Собираем docx
        doc = fill_document(row_dict)

        base_name = f"Заявление ЧСИ на взыскание, {fio}, {iin}"
        docx_path = Path(OUT_FOLDER) / (base_name + ".docx")
        pdf_path  = Path(OUT_FOLDER) / (base_name + ".pdf")

        doc.save(docx_path)

        # 2) Конвертируем в PDF
        convert(str(docx_path), str(pdf_path))

        # 3) Можно удалить промежуточный DOCX, если не нужен
        try:
            os.remove(docx_path)
        except OSError:
            pass

        print(f"Сформирован файл: {pdf_path}")

    wb.close()
    print("Готово.")

if __name__ == "__main__":
    main()


## Полная подача, кроме подписания 

In [ ]:
# -*- coding: utf-8 -*-
# Скрипт:
# 1) Логинится в AISOIP.
# 2) Открывает вкладку "Подача заявления ЧСИ" и жмёт "Подать заявление".
# 3) Для КАЖДОЙ СДЕЛКИ из Excel-списка:
#    - берёт КЛЮЧ из колонки E (ИИН/ЖСН или ФИО) и АДРЕС из колонки I;
#    - находит документ в папке IL_FOLDER НЕ по имени файла, а по ИИН/ЖСН/ФИО из текста:
#         * Исполнительный лист / АТҚАРУ ПАРАҒЫ  -> PDF
#         * Судебный приказ                     -> DOCX
#    - вытаскивает: номер ИД, дату, ИИН/ЖСН должника, ФИО (если есть), суд/регион (если получится);
#    - для выбора ТО/района/ЧСИ использует адрес из Excel (колонка I) как приоритет,
#      если его нет — пытается взять фактический адрес из документа;
#    - заполняет форму:
#         стр.1: ИД, дата, суд, ИИН, ФИО;
#         стр.2: вложение документа и "Заявления";
#         стр.3: ТО, район, ФИО ЧСИ.
#    - жмёт "Далее" три раза.
#
# ВНИМАНИЕ: логин/пароль лучше хранить в env, но оставляю как у вас.

import os
import re
import time
import difflib
from pathlib import Path
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from pypdf import PdfReader
from openpyxl import load_workbook
from docx import Document as DocxDocument


# ================= НАСТРОЙКИ =================
AISOIP_URL = "https://aisoip.adilet.gov.kz/cabinet/exec-productions"
STATEMENTS_URL = "https://aisoip.adilet.gov.kz/cabinet/statements"

LOGIN = "810813301334_230240016634"
PASSWORD = "Qazaq123456*"

# Папка с документами:
# - ИЛ/АТҚАРУ ПАРАҒЫ: PDF
# - Судебный приказ: DOCX
IL_FOLDER = r"C:\Users\User\Desktop\Новая папка\ИЛ"

# Папка с ЗАЯВЛЕНИЯМИ ЧСИ (по ИИН в имени файла)
STATEMENTS_FOLDER = r"C:\Users\User\Desktop\Новая папка\Заявления"

# Excel-список (ключ E + адрес I)
DEALS_XLSX = r"C:\Users\User\Desktop\Новая папка\Список для подачи заявления ЧСИ через АИС ОИП_Omega.xlsx"

# Таблицы судов и ЧСИ
COURTS_XLSX = r"C:\Users\User\Desktop\Новая папка\Суды в АИС ОИП.xlsx"
CHSI_XLSX   = r"C:\Users\User\Desktop\Новая папка\ЧСИ в АИС ОИП.xlsx"

# Пороги совпадения
REGION_MATCH_THRESHOLD = 0.3
COURT_MATCH_THRESHOLD  = 0.3
CHSI_MATCH_THRESHOLD   = 0.25

_t0 = time.time()
def log(msg):
    dt = time.time() - _t0
    print(f"[{dt:7.2f}s] {msg}")


# ================= DEBUG (только ЧСИ) =================
DEBUG_CHSI = True
DEBUG_CHSI_DIR = r"C:\Users\User\Desktop\Новая папка\debug_chsi"

def chsi_dbg_save(driver, name: str):
    if not DEBUG_CHSI:
        return
    try:
        os.makedirs(DEBUG_CHSI_DIR, exist_ok=True)
    except Exception:
        return

    ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    base = os.path.join(DEBUG_CHSI_DIR, f"{ts}_{name}")

    try:
        driver.save_screenshot(base + ".png")
    except Exception as e:
        log(f"[CHSI-DBG] screenshot error: {type(e).__name__}: {e}")

    try:
        with open(base + ".html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
    except Exception as e:
        log(f"[CHSI-DBG] page_source error: {type(e).__name__}: {e}")


class AlreadyRegisteredException(Exception):
    """Исполнительный документ уже зарегистрирован в системе."""
    pass


# =============== Нормализация текста и токены ===============
STOP_WORDS = {
    # русские
    "суд", "суда", "суды", "районный", "районного", "городской",
    "городского", "города", "области", "обл", "обл.", "республики",
    "рк", "республика", "административный", "межрайонный",
    "специализированный", "специализированного", "район", "область",
    "суды", "в", "частные", "судебные", "исполнители", "частных",
    # казахские
    "сот", "соты", "соттары", "облысы", "облыс", "аудандық",
    "қалалық", "қаласы", "облысы",
}

def stem_token(token: str) -> str:
    endings = [
        "ая", "яя", "ой", "ый", "ий", "ое", "ее", "ые", "ие",
        "ого", "его", "ому", "ему", "ыми", "ими", "ых", "их",
        "ую", "юю", "ой", "ей", "ом", "ем", "ам", "ям", "ах", "ях",
    ]
    for e in endings:
        if token.endswith(e) and len(token) > len(e) + 2:
            return token[:-len(e)]
    return token

def normalize_text(s: str) -> str:
    if not s:
        return ""
    s = s.lower()
    s = s.replace("ё", "е").replace("’", "'").replace("`", "'")
    s = re.sub(r"[^0-9a-zа-яәіңғүұқөһі\s]", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokens_without_stopwords(norm: str):
    if not norm:
        return set()
    result = set()
    for t in norm.split():
        if not t:
            continue
        if t in STOP_WORDS:
            continue
        result.add(stem_token(t))
    return result

def extract_12_digits_from_key(key: str):
    if not key:
        return None
    m = re.search(r"\b(\d{12})\b", str(key))
    return m.group(1) if m else None


# =============== Даты (RU месяц словами -> dd.mm.yyyy) ===============
RU_MONTHS = {
    "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
    "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12
}

def parse_ru_date_words(s: str):
    """
    Ищет дату вида "24 сентября 2025 года" -> "24.09.2025"
    """
    if not s:
        return None
    m = re.search(r"\b(\d{1,2})\s+([а-яё]+)\s+(\d{4})\s*(?:года|г\.)?\b", s.lower())
    if not m:
        return None
    day = int(m.group(1))
    mon_word = m.group(2)
    year = int(m.group(3))
    mon = RU_MONTHS.get(mon_word)
    if not mon:
        return None
    try:
        dt = datetime(year, mon, day)
        return dt.strftime("%d.%m.%Y")
    except Exception:
        return None


# =============== Загрузка регионов и судов из Excel ===============
def load_courts_and_regions(xlsx_path: str):
    regions = []
    courts = []

    if not os.path.exists(xlsx_path):
        log(f"[COURTS] Файл судов не найден: {xlsx_path}")
        return regions, courts

    try:
        wb = load_workbook(xlsx_path, read_only=True, data_only=True)
        ws = wb.active
    except Exception as e:
        log(f"[COURTS] Ошибка чтения Excel: {e}")
        return regions, courts

    current_region = None
    region_seen = {}

    for row in ws.iter_rows(min_row=2, values_only=True):
        region_cell, court_cell = row[0], row[1]
        if court_cell is None:
            continue

        if region_cell:
            current_region = str(region_cell).strip()
        if not current_region:
            continue

        if current_region not in region_seen:
            norm_reg = normalize_text(current_region)
            tokens_core_reg = tokens_without_stopwords(norm_reg)
            reg_obj = {
                "region": current_region,
                "norm": norm_reg,
                "tokens_core": tokens_core_reg,
            }
            regions.append(reg_obj)
            region_seen[current_region] = reg_obj

        court_name = str(court_cell).strip()
        norm_court = normalize_text(court_name)
        tokens_core_court = tokens_without_stopwords(norm_court)
        courts.append({
            "court": court_name,
            "region": current_region,
            "norm": norm_court,
            "tokens_core": tokens_core_court,
        })

    wb.close()
    log(f"[COURTS] Регионов: {len(regions)}, судов: {len(courts)}")
    return regions, courts


REGIONS_LIST, COURTS_LIST = load_courts_and_regions(COURTS_XLSX)


# =============== Загрузка таблицы ЧСИ ===============
def load_chsi_table(xlsx_path: str):
    rows = []
    if not os.path.exists(xlsx_path):
        log(f"[CHSI] Файл ЧСИ не найден: {xlsx_path}")
        return rows

    try:
        wb = load_workbook(xlsx_path, read_only=True, data_only=True)
        ws = wb.active
    except Exception as e:
        log(f"[CHSI] Ошибка чтения Excel: {e}")
        return rows

    for row in ws.iter_rows(min_row=2, values_only=True):
        to_cell       = row[0] if len(row) > 0 else None
        district_cell = row[1] if len(row) > 1 else None
        chsi_cell     = row[2] if len(row) > 2 else None

        if not to_cell:
            continue

        to       = str(to_cell).strip()
        district = str(district_cell).strip() if district_cell else ""
        chsi     = str(chsi_cell).strip() if chsi_cell else ""

        norm_to       = normalize_text(to)
        tokens_to     = tokens_without_stopwords(norm_to)
        norm_district = normalize_text(district) if district else ""
        tokens_district = tokens_without_stopwords(norm_district) if district else set()

        rows.append({
            "to": to,
            "district": district,
            "chsi": chsi,
            "norm_to": norm_to,
            "tokens_to": tokens_to,
            "norm_district": norm_district,
            "tokens_district": tokens_district,
        })

    wb.close()
    log(f"[CHSI] Строк в таблице ЧСИ: {len(rows)}")
    return rows


CHSI_ROWS = load_chsi_table(CHSI_XLSX)


# =============== Список сделок: ключ E + адрес I ===============
def load_deals_list(xlsx_path: str):
    """
    Колонка E (индекс 4) = ключ (ИИН/ЖСН или ФИО)
    Колонка I (индекс 8) = адрес (Адрес с СК)
    """
    deals = []
    if not os.path.exists(xlsx_path):
        log(f"[LIST] Файл списка сделок не найден: {xlsx_path}")
        return deals

    try:
        wb = load_workbook(xlsx_path, read_only=True, data_only=True)
        ws = wb.active
    except Exception as e:
        log(f"[LIST] Ошибка чтения файла списка: {e}")
        return deals

    for row in ws.iter_rows(min_row=2, values_only=True):
        if len(row) < 9:
            continue
        key = row[4]   # E
        addr = row[8]  # I
        if not key:
            continue

        deals.append({
            "key": str(key).strip(),
            "excel_address": str(addr).strip() if addr else ""
        })

    wb.close()
    log(f"[LIST] Кол-во записей в списке для подачи: {len(deals)}")
    return deals


DEALS = load_deals_list(DEALS_XLSX)


# =============== Чтение PDF/DOCX: текст и извлечение ИИН/ЖСН/ФИО ===============
def read_pdf_text(pdf_path: str):
    reader = PdfReader(pdf_path)
    parts = []
    for page in reader.pages:
        try:
            t = page.extract_text() or ""
        except Exception:
            t = ""
        if t:
            parts.append(t)
    full_text = "\n".join(parts).replace("\xa0", " ")
    lines = [ln.strip() for ln in full_text.splitlines() if ln.strip()]
    joined = "\n".join(lines)
    return joined, lines

def read_docx_text(docx_path: str):
    doc = DocxDocument(docx_path)
    parts = []
    for p in doc.paragraphs:
        txt = (p.text or "").strip()
        if txt:
            parts.append(txt)
    for tbl in doc.tables:
        for row in tbl.rows:
            for cell in row.cells:
                txt = (cell.text or "").strip()
                if txt:
                    parts.append(txt)
    joined = "\n".join(parts).replace("\xa0", " ")
    lines = [ln.strip() for ln in joined.splitlines() if ln.strip()]
    joined = "\n".join(lines)
    return joined, lines

def extract_iin_or_zsn_from_text(joined_text: str):
    if not joined_text:
        return None
    t = joined_text

    is_kz_exec_sheet = re.search(r"АТҚАРУ\s+ПАРАҒЫ", t, flags=re.IGNORECASE) is not None

    def find_in_block(block_title_regex: str, id_word_regex: str):
        m = re.search(block_title_regex, t, flags=re.IGNORECASE)
        if not m:
            return None
        start = m.end()
        chunk = t[start:start + 2500]
        m_id = re.search(rf"\b({id_word_regex})\b\s*[:\-]?\s*(\d{{12}})\b", chunk, flags=re.IGNORECASE)
        return m_id.group(2) if m_id else None

    if is_kz_exec_sheet:
        iin = find_in_block(r"Жауапкердің\s+толық\s+атауы\s+және\s+мекенжайы", r"ЖСН")
        if iin:
            return iin

    iin = find_in_block(r"Полное\s+наименование\s+ответчика\s+и\s+его\s+адрес", r"ИИН")
    if iin:
        return iin

    iin = find_in_block(r"Ответчик\s*[:\-]", r"ИИН")
    if iin:
        return iin

    m = re.search(r"\bИИН\b\s*[:\-]?\s*(\d{12})\b", t, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    m = re.search(r"\bЖСН\b\s*[:\-]?\s*(\d{12})\b", t, flags=re.IGNORECASE)
    if m:
        return m.group(1)

    return None

def extract_debtor_fio_from_text(joined_text: str):
    if not joined_text:
        return None

    m = re.search(r"\bФИО\s*[:\-]\s*(.+)", joined_text, flags=re.IGNORECASE)
    if m:
        fio_line = m.group(1).splitlines()[0].strip()
        fio_line = re.sub(r"\s{2,}", " ", fio_line).strip(" ,.;")
        if fio_line:
            return fio_line

    m = re.search(r"\bТ\.?\s*А\.?\s*Ә\.?\s*[:\-]\s*(.+)", joined_text, flags=re.IGNORECASE)
    if m:
        fio_line = m.group(1).splitlines()[0].strip()
        fio_line = re.sub(r"\s{2,}", " ", fio_line).strip(" ,.;")
        if fio_line:
            return fio_line

    m = re.search(r"\bОтветчик\s*[:\-]\s*([^\n\r]+)", joined_text, flags=re.IGNORECASE)
    if m:
        fio_line = m.group(1).strip()
        fio_line = re.split(r",\s*ИИН\b|;\s*ИИН\b|\s+ИИН\b", fio_line, maxsplit=1, flags=re.IGNORECASE)[0].strip(" ,.;")
        fio_line = re.sub(r"\s{2,}", " ", fio_line).strip()
        if fio_line:
            return fio_line

    return None


# =============== Индексация документов по ИИН/ЖСН из текста (PDF+DOCX) ===============
def build_docs_index_from_text(folder: str):
    by_iin = {}
    all_docs = []

    if not os.path.exists(folder):
        log(f"[DOCS] Папка не найдена: {folder}")
        return by_iin, all_docs

    pdfs  = list(Path(folder).rglob("*.pdf"))
    docxs = list(Path(folder).rglob("*.docx"))
    log(f"[DOCS] Найдено PDF: {len(pdfs)}, DOCX: {len(docxs)} (включая подпапки)")

    def add_doc(path: str, kind: str, joined_text: str, fio: str, iin: str):
        doc = {
            "path": path,
            "kind": kind,
            "iin": iin,
            "fio_raw": fio,
            "fio_norm": normalize_text(fio) if fio else "",
            "text": joined_text,
        }
        all_docs.append(doc)
        if iin:
            by_iin.setdefault(iin, []).append(doc)

    for p in pdfs:
        path = str(p)
        try:
            joined, _lines = read_pdf_text(path)
        except Exception as e:
            log(f"[DOCS] Не смог прочитать PDF: {path} ({type(e).__name__}: {e})")
            continue
        iin = extract_iin_or_zsn_from_text(joined)
        fio = extract_debtor_fio_from_text(joined)
        add_doc(path, "pdf", joined, fio, iin)

    for p in docxs:
        path = str(p)
        try:
            joined, _lines = read_docx_text(path)
        except Exception as e:
            log(f"[DOCS] Не смог прочитать DOCX: {path} ({type(e).__name__}: {e})")
            continue
        iin = extract_iin_or_zsn_from_text(joined)
        fio = extract_debtor_fio_from_text(joined)
        add_doc(path, "docx", joined, fio, iin)

    log(f"[DOCS] В индексе ИИН/ЖСН: {len(by_iin)} (уникальных)")
    return by_iin, all_docs


DOCS_BY_IIN, DOCS_ALL = build_docs_index_from_text(IL_FOLDER)


def find_doc_for_key(key: str):
    if not key:
        return None

    key_str = str(key).strip()
    if not key_str:
        return None

    key_iin = extract_12_digits_from_key(key_str)
    if key_iin:
        candidates = DOCS_BY_IIN.get(key_iin, [])
        if not candidates:
            return None
        return candidates[0]["path"]

    norm_key = normalize_text(key_str)
    if not norm_key:
        return None

    for doc in DOCS_ALL:
        if doc["fio_norm"] and doc["fio_norm"] == norm_key:
            return doc["path"]

    for doc in DOCS_ALL:
        if doc["fio_norm"] and (norm_key in doc["fio_norm"] or doc["fio_norm"] in norm_key):
            return doc["path"]

    return None


# =============== Поиск региона в тексте ===============
def find_region_in_text(text: str):
    if not REGIONS_LIST:
        return None, 0.0

    norm_text = normalize_text(text)
    if not norm_text:
        return None, 0.0

    text_tokens_core = tokens_without_stopwords(norm_text)

    best = None
    best_score = 0.0

    for r in REGIONS_LIST:
        tokens_core = r["tokens_core"]
        token_score = (len(tokens_core & text_tokens_core) / float(len(tokens_core))) if tokens_core else 0.0
        char_score = difflib.SequenceMatcher(None, r["norm"], norm_text).ratio()
        score = 0.8 * token_score + 0.2 * char_score
        if score > best_score:
            best = r
            best_score = score

    if best and best_score >= REGION_MATCH_THRESHOLD:
        return best["region"], best_score
    return None, best_score


# =============== Строка суда в шапке ===============
def extract_court_header_line(header_lines):
    STAFF_MARKERS = ["секретарь", "заседания", "заседание", "администратора", "администратор", "подписи", "подпись"]
    IGNORE_MARKERS = ["вступлен", "законн", "акт"]

    def has_court_word(text_norm: str) -> bool:
        tokens = text_norm.split()
        court_tokens = {"суд", "суда", "суды", "суде", "суду", "судом", "сот", "соты"}
        return any(t in court_tokens for t in tokens)

    raw_candidates = []
    for i, ln in enumerate(header_lines):
        if not ln.strip():
            continue
        combos = [ln]
        if i > 0:
            combos.append(header_lines[i-1] + " " + ln)
        for combo in combos:
            norm_combo = normalize_text(combo)
            if not norm_combo:
                continue
            if not has_court_word(norm_combo):
                continue
            raw_candidates.append(combo)

    if not raw_candidates:
        return None

    good_candidates = []
    for c in raw_candidates:
        low = c.lower()
        if any(mark in low for mark in STAFF_MARKERS):
            continue
        if any(mark in low for mark in IGNORE_MARKERS):
            continue
        good_candidates.append(c)

    candidates = good_candidates if good_candidates else raw_candidates

    uniq = []
    seen = set()
    for c in candidates:
        if c not in seen:
            uniq.append(c)
            seen.add(c)

    best_line = None
    best_score = -1
    for ln in uniq:
        norm = normalize_text(ln)
        core = tokens_without_stopwords(norm)
        score = len(core)
        if score > best_score:
            best_score = score
            best_line = ln

    return best_line


# =============== Поиск суда в регионе ===============
def find_court_for_region(court_text: str, region_name: str):
    if not court_text or not region_name:
        return None, 0.0

    candidates = [c for c in COURTS_LIST if c["region"] == region_name]
    if not candidates:
        return None, 0.0

    norm_text = normalize_text(court_text)
    text_tokens_core = tokens_without_stopwords(norm_text)

    scored = []
    for c in candidates:
        tokens_core = c["tokens_core"]
        overlap = len(tokens_core & text_tokens_core)
        token_score = (overlap / float(len(tokens_core))) if tokens_core else 0.0
        char_score = difflib.SequenceMatcher(None, c["norm"], norm_text).ratio()
        score = char_score if overlap == 0 else (0.7 * token_score + 0.3 * char_score)
        scored.append({"court": c["court"], "score": score})

    scored.sort(key=lambda x: x["score"], reverse=True)
    best = scored[0] if scored else None

    if best and best["score"] >= COURT_MATCH_THRESHOLD:
        return best["court"], best["score"]
    return None, best["score"] if best else 0.0


# =============== Поиск ТО/района/ЧСИ по адресу ===============
def find_to_chsi_for_address(fact_address: str):
    if not fact_address or not CHSI_ROWS:
        return None, 0.0

    norm_addr = normalize_text(fact_address)
    tokens_addr = tokens_without_stopwords(norm_addr)

    scored = []
    for r in CHSI_ROWS:
        to_tokens   = r["tokens_to"]
        dist_tokens = r["tokens_district"]

        overlap_to   = len(to_tokens & tokens_addr) / float(len(to_tokens)) if to_tokens else 0.0
        overlap_dist = len(dist_tokens & tokens_addr) / float(len(dist_tokens)) if dist_tokens else 0.0

        char_to   = difflib.SequenceMatcher(None, r["norm_to"], norm_addr).ratio()
        char_dist = difflib.SequenceMatcher(None, r["norm_district"], norm_addr).ratio() if r["norm_district"] else 0.0

        score = 0.35 * overlap_to + 0.45 * overlap_dist + 0.10 * char_to + 0.10 * char_dist
        scored.append({"row": r, "score": score})

    scored.sort(key=lambda x: x["score"], reverse=True)
    best = scored[0] if scored else None

    if best and best["score"] >= CHSI_MATCH_THRESHOLD:
        return best["row"], best["score"]
    return None, best["score"] if best else 0.0


# =============== Извлечение данных из PDF или DOCX ===============
def extract_fact_address_from_pdf(joined: str):
    if not joined:
        return None

    block_match = re.search(
        r"Полное наименование ответчика и его адрес:(.+?)(?:Полное наименование|ИСПОЛНИТЕЛЬНЫЙ ЛИСТ|Дата выписки|$)",
        joined,
        flags=re.DOTALL | re.IGNORECASE
    )
    if block_match:
        block_text = block_match.group(1)
        m_fact = re.search(r"Фактический адрес:\s*(.+)", block_text, flags=re.IGNORECASE)
        if m_fact:
            return m_fact.group(1).splitlines()[0].strip()

    m_kz = re.search(r"Нақты\s+мекенжайы\s*:\s*(.+)", joined, flags=re.IGNORECASE)
    if m_kz:
        return m_kz.group(1).splitlines()[0].strip()

    m_fact2 = re.search(r"Фактический адрес:\s*(.+)", joined, flags=re.IGNORECASE)
    if m_fact2:
        return m_fact2.group(1).splitlines()[0].strip()

    return None

def extract_fact_address_from_docx(joined: str):
    if not joined:
        return None

    m = re.search(r"Ответчик\s*:\s*([^\n\r]+)", joined, flags=re.IGNORECASE)
    if m:
        line = m.group(1).strip()
        m_i = re.search(r"\bИИН\b[:\s]*\d{12}\s*,?\s*(.+)$", line, flags=re.IGNORECASE)
        if m_i:
            addr = m_i.group(1).strip()
            if addr:
                return addr

    m2 = re.search(r"Фактический адрес:\s*(.+)", joined, flags=re.IGNORECASE)
    if m2:
        return m2.group(1).splitlines()[0].strip()

    m3 = re.search(r"Юридический адрес:\s*(.+)", joined, flags=re.IGNORECASE)
    if m3:
        return m3.group(1).splitlines()[0].strip()

    return None

def extract_data_from_pdf(pdf_path: str):
    log(f"[PDF] Читаю файл: {pdf_path}")
    joined, lines = read_pdf_text(pdf_path)
    header_lines = lines[:40]
    header_text  = "\n".join(header_lines)

    fact_address = extract_fact_address_from_pdf(joined)
    if fact_address:
        log(f"[PDF] Фактический адрес (fallback): {fact_address}")
    else:
        log("[PDF] Фактический адрес не найден (fallback).")

    num = None
    m_num = re.search(r"^[^\S\r\n]*№\s*([0-9][0-9\-–/]*\/[0-9\-–/]+)", joined, flags=re.MULTILINE)
    if not m_num:
        m_num = re.search(r"ИСПОЛНИТЕЛЬНЫЙ\s+ЛИСТ.*?№\s*([0-9][0-9\-–/]+)", joined, flags=re.DOTALL | re.IGNORECASE)
    if not m_num:
        m_num = re.search(r"№\s*([0-9][0-9\-–/]+)", joined)
    if m_num:
        num = m_num.group(1).strip()

    date_str = None
    m_date = re.search(r"Дата выписки\s+(\d{2}\.\d{2}\.\d{4})", joined, flags=re.IGNORECASE)
    if m_date:
        date_str = m_date.group(1)
    else:
        m_date2 = re.search(r"(Шығарылу\s+күні|Шыгарылу\s+күні)\s+(\d{2}\.\d{2}\.\d{4})", joined, flags=re.IGNORECASE)
        if m_date2:
            date_str = m_date2.group(2)

    iin = extract_iin_or_zsn_from_text(joined)
    fio = extract_debtor_fio_from_text(joined)

    court_region, reg_score = find_region_in_text(header_text)
    court_header_line = extract_court_header_line(header_lines)
    court_name = None

    if court_region:
        log(f"[PDF] Регион по шапке (score={reg_score:.2f}): {court_region}")
    else:
        log(f"[PDF] Регион по шапке не определён (score={reg_score:.2f})")

    if court_header_line:
        log(f"[PDF] Строка суда (шапка): '{court_header_line}'")
        if court_region:
            court_name, court_score = find_court_for_region(court_header_line, court_region)
            if court_name:
                log(f"[PDF] Суд по шапке (score={court_score:.2f}): {court_name}")
    else:
        log("[PDF] Строка суда в шапке не найдена")

    if not all([num, date_str, iin]):
        log(f"[PDF] ВНИМАНИЕ: нет номера/даты/ИИН(ЖСН) в {pdf_path}")
    else:
        log(f"[PDF] Найдено: номер={num}, дата={date_str}, ИИН/ЖСН={iin}, ФИО={fio or 'не найдено'}")

    return {
        "number":       num,
        "date":         date_str,
        "iin":          iin,
        "fio":          fio,
        "court_name":   court_name,
        "court_region": court_region,
        "fact_address": fact_address,
        "doc_kind":     "pdf",
    }

def extract_data_from_docx(docx_path: str):
    log(f"[DOCX] Читаю файл: {docx_path}")
    joined, lines = read_docx_text(docx_path)
    header_lines = lines[:50]
    header_text  = "\n".join(header_lines)

    fact_address = extract_fact_address_from_docx(joined)
    if fact_address:
        log(f"[DOCX] Адрес (fallback): {fact_address}")
    else:
        log("[DOCX] Адрес не найден (fallback).")

    num = None
    m_num = re.search(r"№\s*([0-9][0-9\-–/]+)", joined)
    if m_num:
        num = m_num.group(1).strip()

    date_str = None
    m_date = re.search(r"\b(\d{2}\.\d{2}\.\d{4})\b", header_text)
    if m_date:
        date_str = m_date.group(1)
    else:
        date_str = parse_ru_date_words(header_text)
        if not date_str:
            date_str = parse_ru_date_words(joined)

    iin = extract_iin_or_zsn_from_text(joined)
    fio = extract_debtor_fio_from_text(joined)

    court_region, reg_score = find_region_in_text(header_text)
    court_header_line = extract_court_header_line(header_lines)
    court_name = None

    if court_region:
        log(f"[DOCX] Регион по шапке (score={reg_score:.2f}): {court_region}")
    else:
        log(f"[DOCX] Регион по шапке не определён (score={reg_score:.2f})")

    if court_header_line:
        log(f"[DOCX] Строка суда (шапка): '{court_header_line}'")
        if court_region:
            court_name, court_score = find_court_for_region(court_header_line, court_region)
            if court_name:
                log(f"[DOCX] Суд по шапке (score={court_score:.2f}): {court_name}")
    else:
        m_c = re.search(r"Судья\s*:\s*([^\n\r]+)", joined, flags=re.IGNORECASE)
        if m_c:
            court_header_line = m_c.group(1).strip()
            log(f"[DOCX] Строка суда (из 'Судья:'): '{court_header_line}'")
            if court_region:
                court_name, court_score = find_court_for_region(court_header_line, court_region)
                if court_name:
                    log(f"[DOCX] Суд (score={court_score:.2f}): {court_name}")

    if not all([num, date_str, iin]):
        log(f"[DOCX] ВНИМАНИЕ: нет номера/даты/ИИН в {docx_path}")
    else:
        log(f"[DOCX] Найдено: номер={num}, дата={date_str}, ИИН={iin}, ФИО={fio or 'не найдено'}")

    return {
        "number":       num,
        "date":         date_str,
        "iin":          iin,
        "fio":          fio,
        "court_name":   court_name,
        "court_region": court_region,
        "fact_address": fact_address,
        "doc_kind":     "docx",
    }

def extract_data_from_document(path: str):
    ext = Path(path).suffix.lower()
    if ext == ".pdf":
        return extract_data_from_pdf(path)
    if ext == ".docx":
        return extract_data_from_docx(path)
    raise ValueError(f"Неподдерживаемый тип файла: {ext}")


# =============== Selenium: драйвер + логин ===============
def make_driver():
    opts = webdriver.ChromeOptions()
    prefs = {
        "safebrowsing.enabled": True,
        "profile.managed_default_content_settings.images": 2,
    }
    opts.add_experimental_option("prefs", prefs)
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")

    service = Service(ChromeDriverManager().install())
    drv = webdriver.Chrome(service=service, options=opts)
    log("[INIT] Запущен Chrome WebDriver")
    return drv

def fill_login_form(driver):
    wait = WebDriverWait(driver, 20)
    user = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR,
        "input[name='usernameUserInput'], input#usernameUserInput, "
        "input[name='username'], input#username, form#loginForm input[type='text']"
    )))
    pwd = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR,
        "input#password, input[name='password'], form#loginForm input[type='password']"
    )))

    driver.execute_script("""
        const u = arguments[0], p = arguments[1], lu = arguments[2], lp = arguments[3];
        u.value = lu; u.dispatchEvent(new Event('input', {bubbles:true})); u.dispatchEvent(new Event('change', {bubbles:true}));
        p.value = lp; p.dispatchEvent(new Event('input', {bubbles:true})); p.dispatchEvent(new Event('change', {bubbles:true}));
    """, user, pwd, LOGIN, PASSWORD)

    try:
        user.clear()
        user.send_keys(LOGIN)
    except Exception:
        pass
    try:
        pwd.clear()
        pwd.send_keys(PASSWORD)
    except Exception:
        pass

def click_sign_in(driver):
    wait = WebDriverWait(driver, 10)
    btn = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "#signInButton, form#loginForm button.ui.primary.button.fluid, button[type='submit']"
    )))
    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    except Exception:
        pass

    for _ in range(3):
        try:
            btn.click()
        except Exception:
            pass
        time.sleep(0.15)
        try:
            driver.execute_script("arguments[0].click();", btn)
        except Exception:
            pass
        time.sleep(0.2)

def is_logged_in(driver):
    try:
        url = driver.current_url or ""
        return "/cabinet/" in url
    except Exception:
        return False

def login_with_retries(driver, max_rounds=6):
    log("[LOGIN] Начинаю авторизацию…")
    driver.get(AISOIP_URL)

    for i in range(1, max_rounds + 1):
        log(f"[LOGIN] попытка {i}/{max_rounds}")
        try:
            WebDriverWait(driver, 25).until(EC.presence_of_element_located((
                By.CSS_SELECTOR,
                "input[name='usernameUserInput'], input#usernameUserInput, "
                "input[name='username'], input#username, form#loginForm input[type='text']"
            )))
        except TimeoutException:
            log("[LOGIN] Не нашли форму логина, обновляю страницу…")
            try:
                driver.refresh()
            except Exception:
                pass
            time.sleep(1.0)
            continue

        fill_login_form(driver)
        time.sleep(0.2)
        click_sign_in(driver)

        try:
            WebDriverWait(driver, 15).until(lambda d: is_logged_in(d))
            if is_logged_in(driver):
                log("[OK] Авторизация выполнена.")
                return
        except TimeoutException:
            log("[LOGIN] Похоже, не вошли, пробуем ещё раз…")

        try:
            driver.refresh()
        except Exception:
            pass
        time.sleep(1.0)

    raise TimeoutException("Не удалось войти после серии повторов.")


# =============== Переход в «Подача заявления ЧСИ» ===============
def open_statements_tab(driver):
    log("[NAV] Открываю вкладку 'Подача заявления ЧСИ'")
    driver.get(STATEMENTS_URL)
    wait = WebDriverWait(driver, 30)

    try:
        link = wait.until(EC.presence_of_element_located((
            By.XPATH,
            "//a[contains(@class,'navigation-item') and contains(.,'Подача заявления ЧСИ')]"
        )))
        try:
            driver.execute_script("arguments[0].click();", link)
        except Exception:
            pass
    except Exception:
        pass

    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(.,'Сформированные заявления')]")))
    except Exception:
        wait.until(EC.url_contains("/cabinet/statements"))

    log("[NAV] Вкладка 'Подача заявления ЧСИ' открыта.")

def click_podat_zayavlenie(driver):
    log("[NAV] Жму кнопку 'Подать заявление'")
    wait = WebDriverWait(driver, 30)

    xpath_btn = (
        "//span[contains(@class,'v-btn__content') and normalize-space()='Подать заявление']"
        "/ancestor::*[self::a or self::button][1]"
    )
    btn = wait.until(EC.presence_of_element_located((By.XPATH, xpath_btn)))

    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    except Exception:
        pass

    try:
        driver.execute_script("arguments[0].click();", btn)
    except Exception:
        btn.click()

    wait.until(EC.url_contains("/cabinet/statements/new"))
    log("[NAV] Форма 'Подать заявление' открыта.")


# =============== Заполнение формы (страница 1) ===============
def fill_statement_form(driver, data):
    wait    = WebDriverWait(driver, 20)
    actions = ActionChains(driver)

    log("[FORM] Выбираю 'Источник ИД' = АИАС «Төрелік»")
    src_input = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//label[contains(.,'Источник ИД')]/following::input[1]"
    )))
    src_input.click()
    time.sleep(0.3)

    src_item = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//div[contains(@class,'v-list-item__title') and normalize-space()='АИАС «Төрелік»']"
    )))
    try:
        src_item.click()
    except Exception:
        driver.execute_script("arguments[0].click();", src_item)

    if data.get("number"):
        log("[FORM] Заполняю 'Номер исполнительного документа'")
        num_input = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            "//label[contains(.,'Номер исполнительного документа')]/following::input[1]"
        )))
        num_input.click()
        actions.key_down('\ue009').send_keys('a').key_up('\ue009').send_keys('\ue003').perform()
        num_input.send_keys(data["number"])

    if data.get("date"):
        log("[FORM] Заполняю 'Дата выписки исполнительного документа'")
        date_input = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            "//label[contains(.,'Дата выписки исполнительного документа')]/following::input[1]"
        )))
        date_input.click()
        actions.key_down('\ue009').send_keys('a').key_up('\ue009').send_keys('\ue003').perform()
        date_input.send_keys(data["date"])

    court_region = data.get("court_region")
    court_name   = data.get("court_name")
    if court_region and court_name:
        log(f"[FORM] Выбираю суд: {court_region} → {court_name}")
        court_input = wait.until(EC.presence_of_element_located((
            By.XPATH,
            "//label[contains(.,'Орган, выдавший исполнительный документ')]/following::input[1]"
        )))
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", court_input)
            time.sleep(0.3)
        except Exception:
            pass

        try:
            driver.execute_script("arguments[0].click();", court_input)
        except Exception:
            try:
                court_input.click()
            except Exception:
                log("[FORM] Не удалось кликнуть по полю суда, продолжаю…")

        time.sleep(0.5)

        region_xpath = (
            "//div[contains(@class,'v-list-item__title') and contains(normalize-space(), '{}')]"
        ).format(court_region)
        region_el = wait.until(EC.element_to_be_clickable((By.XPATH, region_xpath)))
        try:
            driver.execute_script("arguments[0].click();", region_el)
        except Exception:
            region_el.click()
        time.sleep(0.4)

        court_xpath = (
            "//div[contains(@class,'v-list-item__title') and contains(normalize-space(), '{}')]"
        ).format(court_name)
        court_el = wait.until(EC.element_to_be_clickable((By.XPATH, court_xpath)))
        try:
            driver.execute_script("arguments[0].click();", court_el)
        except Exception:
            court_el.click()
    else:
        log("[FORM] Суд не определён, выбор суда пропускаю!")

    if data.get("iin"):
        log("[FORM] Заполняю 'ИИН/БИН должника'")
        iin_input = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            "//label[contains(.,'ИИН/БИН должника')]/following::input[1]"
        )))
        iin_input.click()
        actions.key_down('\ue009').send_keys('a').key_up('\ue009').send_keys('\ue003').perform()
        iin_input.send_keys(data["iin"])

    if data.get("fio"):
        log("[FORM] Заполняю 'ФИО/Наименование должника'")
        fio_input = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            "//label[contains(.,'ФИО/Наименование должника')]/following::input[1]"
        )))
        fio_input.click()
        actions.key_down('\ue009').send_keys('a').key_up('\ue009').send_keys('\ue003').perform()
        fio_input.send_keys(data["fio"])

def click_next(driver, label="Далее"):
    log(f"[FORM] Жму кнопку '{label}'")
    btn = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@id='next-btn' or .//span[normalize-space()='Далее']]"))
    )
    try:
        btn.click()
    except Exception:
        driver.execute_script("arguments[0].click();", btn)
    time.sleep(2.0)


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time


def fill_to_and_chsi(
    driver,
    to_name: str,
    district_name: str,
    chsi_name: str,
    timeout: int = 45,
    DEBUG_CHSI: bool = False,
    chsi_dbg_save=None,
    log=print,
    click_next_btn: bool = True,
    max_retries_per_field: int = 3,
) -> bool:
    """
    Надёжный выбор в блоке "Выберите ЧСИ" (Vuetify v-select):
      - Выберите ТО
      - Выберите Район
      - ФИО ЧСИ
    + умеет: (а) открывать меню разными способами, (б) выбирать через клик,
            (в) при необходимости печатать в input (если в вашем Vuetify есть фильтр),
            (г) проверять, что value реально установился,
            (д) не падать, если район/ЧСИ уже автоподтянулись после ТО.

    Возвращает True если всё выбрано (и, опционально, нажато "Далее"), иначе False.
    """

    wait = WebDriverWait(driver, timeout)

    def norm(s: str) -> str:
        return " ".join((s or "").replace("\u00a0", " ").split()).strip().upper()

    def scroll_center(el):
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        except Exception:
            pass
        time.sleep(0.15)

    def js_click(el) -> bool:
        try:
            driver.execute_script("arguments[0].click();", el)
            return True
        except Exception:
            try:
                el.click()
                return True
            except Exception:
                return False

    def wrapper_by_label(label_text: str):
        # Берём ближайший v-input по label (самый стабильный якорь)
        return wait.until(EC.presence_of_element_located((
            By.XPATH,
            f"//label[contains(normalize-space(),'{label_text}')]/ancestor::div[contains(@class,'v-input')][1]"
        )))

    def get_input(wrapper):
        # В v-select обычно есть input, даже если readonly
        try:
            return wrapper.find_element(By.CSS_SELECTOR, "input")
        except Exception:
            return None

    def read_value_by_label(label_text: str) -> str:
        try:
            wrap = wrapper_by_label(label_text)
            inp = get_input(wrap)
            return (inp.get_attribute("value") or "").strip() if inp else ""
        except Exception:
            return ""

    def close_any_open_menu():
        # Закрыть меню на всякий случай
        try:
            driver.find_element(By.TAG_NAME, "body").click()
        except Exception:
            pass
        time.sleep(0.15)

    def open_menu(wrapper):
        """
        Открывает выпадающий список v-select.
        Делает несколько попыток разными кликами.
        """
        scroll_center(wrapper)

        # 1) клик по input (обычно самое надёжное)
        inp = get_input(wrapper)
        if inp and js_click(inp):
            time.sleep(0.2)
            return

        # 2) по слотам/иконкам
        for css in [".v-select__slot", ".v-input__slot", ".v-input__append-inner", ".v-input__icon--append"]:
            try:
                el = wrapper.find_element(By.CSS_SELECTOR, css)
                if js_click(el):
                    time.sleep(0.2)
                    return
            except Exception:
                continue

        # 3) общий клик по wrapper
        js_click(wrapper)
        time.sleep(0.2)

    def get_active_menu():
        # Активное меню Vuetify (важно: именно active)
        menu = wait.until(EC.presence_of_element_located((
            By.CSS_SELECTOR,
            "div.v-menu__content.menuable__content__active"
        )))
        wait.until(lambda d: menu.is_displayed())
        return menu

    def menu_items(menu):
        # Пункты списка (варианты)
        items = menu.find_elements(By.CSS_SELECTOR, "div[role='option'] .v-list-item__title")
        if not items:
            items = menu.find_elements(By.CSS_SELECTOR, ".v-list-item__title")
        if not items:
            items = menu.find_elements(By.CSS_SELECTOR, ".v-list-item")
        return items

    def click_option(menu, option_text: str) -> bool:
        target = norm(option_text)
        if not target:
            return False

        items = menu_items(menu)

        # 1) exact
        for it in items:
            txt = norm(getattr(it, "text", "") or "")
            if txt == target:
                js_click(it)
                time.sleep(0.35)
                return True

        # 2) contains
        for it in items:
            txt = norm(getattr(it, "text", "") or "")
            if target in txt and txt:
                js_click(it)
                time.sleep(0.35)
                return True

        return False

    def try_type_filter(wrapper, option_text: str):
        """
        Если в вашем v-select есть фильтр по вводу (иногда input принимает текст),
        попробуем ввести текст и потом выбрать.
        Безопасно: если readonly — просто ничего не произойдёт.
        """
        inp = get_input(wrapper)
        if not inp:
            return
        try:
            inp.clear()
        except Exception:
            pass
        try:
            inp.send_keys(option_text)
            time.sleep(0.25)
        except Exception:
            pass

    def select_by_label(label_text: str, option_text: str) -> bool:
        """
        Пытается выбрать option_text в поле label_text.
        Делает несколько попыток: открыть меню -> кликнуть пункт.
        Если не нашло — пробует напечатать в input (если фильтр есть) и снова искать.
        """
        target = (option_text or "").strip()
        if not target:
            return True  # ничего выбирать

        for attempt in range(1, max_retries_per_field + 1):
            try:
                close_any_open_menu()
                wrap = wrapper_by_label(label_text)
                open_menu(wrap)

                menu = get_active_menu()
                ok = click_option(menu, target)
                if ok:
                    return True

                # если не нашли — попробуем фильтр вводом (если поддерживается)
                try_type_filter(wrap, target)
                time.sleep(0.2)

                # меню могло обновиться — возьмём снова
                try:
                    menu = get_active_menu()
                except Exception:
                    open_menu(wrap)
                    menu = get_active_menu()

                ok = click_option(menu, target)
                if ok:
                    return True

                log(f"[FORM] '{label_text}': не нашёл '{target}' (попытка {attempt}/{max_retries_per_field})")

            except TimeoutException:
                log(f"[FORM] '{label_text}': timeout при открытии меню (попытка {attempt}/{max_retries_per_field})")
                if DEBUG_CHSI and chsi_dbg_save:
                    chsi_dbg_save(driver, f"timeout_open_menu_{label_text}_try{attempt}")
            except Exception as e:
                log(f"[FORM] '{label_text}': ошибка {type(e).__name__}: {e} (попытка {attempt}/{max_retries_per_field})")
                if DEBUG_CHSI and chsi_dbg_save:
                    chsi_dbg_save(driver, f"error_select_{label_text}_try{attempt}")

            time.sleep(0.4)

        return False

    def value_matches(label_text: str, expected: str) -> bool:
        v = norm(read_value_by_label(label_text))
        e = norm(expected)
        if not e:
            return True
        if v == e:
            return True
        # иногда в поле отображается сокращённо/по-другому — тогда хотя бы contains
        return e in v if v else False

    # ---------------- START ----------------
    try:
        # ТО
        if to_name:
            log(f"[FORM] Выбираю ТО: {to_name}")
            if not select_by_label("Выберите ТО", to_name):
                log("[FORM] ТО: НЕ выбран (нет совпадения в списке).")
                if DEBUG_CHSI and chsi_dbg_save:
                    chsi_dbg_save(driver, "to_select_fail")
                return False

            # дать подгрузить зависимые списки
            time.sleep(1.0)

        # Район (иногда после ТО он уже подставлен автоматически — тогда не трогаем)
        if district_name:
            current_district = read_value_by_label("Выберите Район")
            if current_district and value_matches("Выберите Район", district_name):
                log(f"[FORM] Район уже выбран: '{current_district}'")
            else:
                log(f"[FORM] Выбираю Район: {district_name}")
                if not select_by_label("Выберите Район", district_name):
                    log("[FORM] Район: НЕ выбран (нет совпадения в списке).")
                    if DEBUG_CHSI and chsi_dbg_save:
                        chsi_dbg_save(driver, "district_select_fail")
                    return False
                time.sleep(1.0)

        # ЧСИ (аналогично: может подставиться автоматически)
        if chsi_name:
            current_chsi = read_value_by_label("ФИО ЧСИ")
            if current_chsi and value_matches("ФИО ЧСИ", chsi_name):
                log(f"[FORM] ЧСИ уже выбран: '{current_chsi}'")
            else:
                log(f"[FORM] Выбираю ЧСИ: {chsi_name}")
                if not select_by_label("ФИО ЧСИ", chsi_name):
                    log("[FORM] ЧСИ: НЕ выбран (нет совпадения в списке).")
                    if DEBUG_CHSI and chsi_dbg_save:
                        chsi_dbg_save(driver, "chsi_select_fail")
                    return False

                # проверим что реально установилось
                time.sleep(0.3)
                if not value_matches("ФИО ЧСИ", chsi_name):
                    value_now = read_value_by_label("ФИО ЧСИ")
                    log(f"[FORM] ЧСИ: значение не установилось. ожидаю='{chsi_name}', сейчас='{value_now}'")
                    if DEBUG_CHSI and chsi_dbg_save:
                        chsi_dbg_save(driver, "chsi_value_not_set")
                    return False

                log(f"[FORM] ЧСИ выбран: '{read_value_by_label('ФИО ЧСИ')}'")

        # Далее (опционально)
        if click_next_btn:
            log("[FORM] Жму кнопку 'Далее (ТО/район/ЧСИ)'")
            btn_next = wait.until(EC.element_to_be_clickable((By.ID, "next-btn")))
            scroll_center(btn_next)
            if not js_click(btn_next):
                log("[FORM] Не удалось нажать 'Далее' (next-btn).")
                if DEBUG_CHSI and chsi_dbg_save:
                    chsi_dbg_save(driver, "next_btn_click_fail")
                return False
            time.sleep(0.8)

        return True

    except TimeoutException:
        log("[FORM] TimeoutException в fill_to_and_chsi (не дождались элемента/меню).")
        if DEBUG_CHSI and chsi_dbg_save:
            chsi_dbg_save(driver, "fill_to_and_chsi_timeout")
        return False

    except Exception as e:
        log(f"[FORM] Ошибка в fill_to_and_chsi: {type(e).__name__}: {e}")
        if DEBUG_CHSI and chsi_dbg_save:
            chsi_dbg_save(driver, "fill_to_and_chsi_error")
        return False



# =============== Загрузка файлов (страница 2) ===============
def find_zayavlenie_file(iin: str):
    if not iin:
        return None
    folder = Path(STATEMENTS_FOLDER)
    if not folder.exists():
        log(f"[UPLOAD] Папка с заявлениями не найдена: {STATEMENTS_FOLDER}")
        return None

    for f in folder.glob("*.pdf"):
        if iin in f.name:
            return str(f)
    return None

def upload_files(driver, doc_path: str, iin: str):
    wait = WebDriverWait(driver, 20)
    log("[UPLOAD] Прикрепляю исполнительный документ (ИЛ/СП)")
    try:
        exec_input = wait.until(EC.presence_of_element_located((By.XPATH, "(//input[@type='file'])[1]")))
    except TimeoutException:
        try:
            banners = driver.find_elements(By.XPATH, "//*[contains(.,'Данный исполнительный документ уже зарегистрирован в Системе')]")
        except Exception:
            banners = []
        if banners:
            log("[UPLOAD] ИД уже зарегистрирован — пропускаю.")
            raise AlreadyRegisteredException()
        raise

    exec_input.send_keys(doc_path)
    time.sleep(1.0)

    other_path = find_zayavlenie_file(iin)
    if not other_path:
        log(f"[UPLOAD] Не найдено 'Заявление ...' по ИИН {iin} — второе поле пропускаю.")
        return

    log(f"[UPLOAD] Прикрепляю 'Заявление ЧСИ': {other_path}")
    other_input = wait.until(EC.presence_of_element_located((By.XPATH, "(//input[@type='file'])[2]")))
    other_input.send_keys(other_path)
    time.sleep(1.0)


# =============== MAIN ===============
def main():
    driver = make_driver()
    try:
        login_with_retries(driver)
        open_statements_tab(driver)

        deals = DEALS
        log(f"[MAIN] В списке для подачи: {len(deals)} записей")

        for deal in deals:
            key = deal["key"]
            excel_address = (deal.get("excel_address") or "").strip()

            doc_path = find_doc_for_key(key)
            if not doc_path:
                log(f"[MAIN] Для ключа '{key}' не найден документ (PDF/DOCX) в папке ИЛ (по тексту), пропускаю.")
                continue

            log(f"[MAIN] Обрабатываю ключ '{key}', Документ: {doc_path}")

            try:
                data = extract_data_from_document(doc_path)

                if not (data.get("number") and data.get("date") and data.get("iin")):
                    log("[MAIN] Нет номера/даты/ИИН(ЖСН) в документе — пропускаю.")
                    continue

                if excel_address:
                    data["fact_address"] = excel_address
                    log(f"[MAIN] Адрес для ТО/ЧСИ взят из Excel (колонка I): {excel_address}")
                else:
                    if data.get("fact_address"):
                        log(f"[MAIN] Адрес для ТО/ЧСИ взят из документа (fallback): {data['fact_address']}")
                    else:
                        log("[MAIN] ВНИМАНИЕ: адрес не найден ни в Excel, ни в документе.")

                open_statements_tab(driver)
                click_podat_zayavlenie(driver)

                fill_statement_form(driver, data)
                click_next(driver, label="Далее (страница 1)")

                upload_files(driver, doc_path, data["iin"])
                click_next(driver, label="Далее (страница 2)")

                # --- поиск ТО/района/ЧСИ по адресу ---
                row_chsi, score = find_to_chsi_for_address(data.get("fact_address") or "")
                if not row_chsi:
                    raise Exception(f"Не удалось определить ТО/район/ЧСИ по адресу (score={score:.2f}): {data.get('fact_address')}")

                to_name = row_chsi.get("to", "")
                district_name = row_chsi.get("district", "")
                chsi_name = row_chsi.get("chsi", "")

                log(f"[CHSI] Подбор по адресу (score={score:.2f}): ТО='{to_name}', район='{district_name}', ЧСИ='{chsi_name}'")

                ok = fill_to_and_chsi(
                    driver,
                    to_name,
                    district_name,
                    chsi_name,
                    timeout=25,
                    DEBUG_CHSI=DEBUG_CHSI,
                    chsi_dbg_save=chsi_dbg_save,
                    log=log,
                )
                if not ok:
                    raise Exception("Не удалось выбрать ТО/район/ЧСИ на странице")

                log(f"[MAIN] Заявление по документу {doc_path} обработано.\n")

            except AlreadyRegisteredException:
                log(f"[MAIN] ИД по ключу '{key}' уже зарегистрирован. Следующая запись.\n")
                continue
            except Exception as e:
                log(f"[MAIN] ОШИБКА при обработке ключа '{key}': {type(e).__name__}: {e}. Пропускаю.\n")
                continue

        log("[MAIN] Все записи обработаны. Оставляю браузер открытым.")
        input("Нажмите Enter, чтобы закрыть браузер и завершить скрипт...")

    finally:
        driver.quit()
        log("[EXIT] Браузер закрыт.")


if __name__ == "__main__":
    main()


[   0.02s] [COURTS] Регионов: 20, судов: 279
[   0.02s] [CHSI] Строк в таблице ЧСИ: 23
[   0.03s] [LIST] Кол-во записей в списке для подачи: 2
[   0.03s] [DOCS] Найдено PDF: 2, DOCX: 0 (включая подпапки)
[   9.31s] [DOCS] В индексе ИИН/ЖСН: 2 (уникальных)
[  11.55s] [INIT] Запущен Chrome WebDriver
[  11.55s] [LOGIN] Начинаю авторизацию…
[  12.43s] [LOGIN] попытка 1/6
[  18.97s] [OK] Авторизация выполнена.
[  18.97s] [NAV] Открываю вкладку 'Подача заявления ЧСИ'
[  19.32s] [NAV] Вкладка 'Подача заявления ЧСИ' открыта.
[  19.32s] [MAIN] В списке для подачи: 2 записей
[  19.32s] [MAIN] Обрабатываю ключ '060427600795', Документ: C:\Users\User\Desktop\Новая папка\ИЛ\ТЕМИРОВА СЕВИНЧ КАБИЛҚЫЗЫ.pdf
[  19.32s] [PDF] Читаю файл: C:\Users\User\Desktop\Новая папка\ИЛ\ТЕМИРОВА СЕВИНЧ КАБИЛҚЫЗЫ.pdf
[  19.65s] [PDF] Фактический адрес (fallback): КАЗАХСТАН, КАРАГАНДИНСКАЯ, КАРАГАНДА, имени
[  19.66s] [PDF] Регион по шапке (score=0.80): Суды в г.Алматы
[  19.66s] [PDF] Строка суда (шапка): '№ 7517-25-0